In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import time
from functools import partial

import numpy as np
from labcore.measurement import *

Problem: we have an instrument that returns *delayed* data, i.e., we write a function that starts an experiment on the instrument but does not return the data.
The data needs to be gathered from the instrument independently afterwards and asynchronously. 
How to incorporate that into our measurement framework?

In [8]:
global_start = None
global_stop = None
global_delay = None


def dummy_collector(*specs, batchsize=1):
    """A generator that simulates collecting data.
    yields for each spec a different power of the current number.
    """
    global global_start
    global global_stop
    global global_delay
    
    nspecs = len(specs)
    buffer = []
    
    for x in np.arange(global_start, global_stop):
        time.sleep(global_delay)
        buffer.append(tuple([x**i for i in range(1, nspecs+1)]))
        
        if len(buffer) >= batchsize:
            print(f'releasing {len(buffer)} records.')
            for b in buffer:
                yield b
            buffer = []

    print(f'releasing {len(buffer)} records.')
    for b in buffer:
        yield b


def background_recording(*specs):
    """
    returns a decorator. the decorated function will, when called,
    return a sweep that calls the decorated function once, followed by a Sweep of `gather_data`.
    The output of `gather_data` is recorded as specified by the specs that
    are passed to this function to create the generator.
    """
    
    def decorator(fun):
        
        def sweep(collector, **collector_kwargs):
            """Return a sweep that executes the decorated setup function and then iterates over
            gathered data."""
            setup_sweep = once(fun)
            gather_sweep = Sweep(record_as(collector(*specs, **collector_kwargs), *specs))
            return setup_sweep + gather_sweep
        
        return sweep
        
    return decorator


def create_background_sweep(decorated_setup_function, collector, **collector_kwargs):
    sweep = decorated_setup_function(collector, **collector_kwargs)
    return sweep


# Note: this decoration will give us a function that creates a sweep when called.
# we call it when creating the sweep to avoid the sweep getting exhausted.
@background_recording(
    indep('a'), dep('b'), dep('c')
)
def start_my_experiment(start=1, stop=10, delay=0.1):
    global global_start
    global global_stop
    global global_delay
    
    print(f'my parameters: start={start}, stop={stop}, delay={delay}')
    
    global_start = start
    global_stop = stop
    global_delay = delay

In [10]:
sweep = create_background_sweep(start_my_experiment, dummy_collector, batchsize=3)

sweep.set_action_opts(
    start_my_experiment=dict(delay=0.2, start=3)
)

for ret in sweep:
    print(ret)

my parameters: start=3, stop=10, delay=0.2
{'a': None, 'b': None, 'c': None}
releasing 3 records.
{'a': 3, 'b': 9, 'c': 27}
{'a': 4, 'b': 16, 'c': 64}
{'a': 5, 'b': 25, 'c': 125}
releasing 3 records.
{'a': 6, 'b': 36, 'c': 216}
{'a': 7, 'b': 49, 'c': 343}
{'a': 8, 'b': 64, 'c': 512}
releasing 1 records.
{'a': 9, 'b': 81, 'c': 729}


In [13]:
create_dummy_sweep = partial(create_background_sweep, collector=dummy_collector)

sweep = create_dummy_sweep(start_my_experiment, batchsize=4)
sweep.set_action_opts(
    start_my_experiment=dict(delay=0.2, start=1)
)

for ret in sweep:
    print(ret)

my parameters: start=1, stop=10, delay=0.2
{'a': None, 'b': None, 'c': None}
releasing 4 records.
{'a': 1, 'b': 1, 'c': 1}
{'a': 2, 'b': 4, 'c': 8}
{'a': 3, 'b': 9, 'c': 27}
{'a': 4, 'b': 16, 'c': 64}
releasing 4 records.
{'a': 5, 'b': 25, 'c': 125}
{'a': 6, 'b': 36, 'c': 216}
{'a': 7, 'b': 49, 'c': 343}
{'a': 8, 'b': 64, 'c': 512}
releasing 1 records.
{'a': 9, 'b': 81, 'c': 729}
